# Chavatik Thorarit 6610110066 Sec02

In [ ]:
# 1. โปรดตั้งค่า directory ที่ใช้ทำงาน
# 2. สร้างและเปิดใช้งาน environment
import Pkg
Pkg.activate(".")                               # . คือ directory ปัจจุบัน
Pkg.instantiate()                               # สร้างไฟล์ Project.toml และ Manifest.toml
Pkg.add(["Plots", "DataFrames", "CSV", "StatsPlots", "StatsBase", "Random"])         # เพิ่ม packages ลงใน environment
Pkg.status()                                    # ตรวจสอบ packages ที่ติดตั้ง
# โหลด packages ด้วยคำสั่ง using
using Plots, DataFrames, CSV, StatsPlots, StatsBase, Random

# 1. โหลด dataset

In [ ]:
# โหลดข้อมูล personality_dataset (personality_dataset.csv) ลงใน DataFrames
df = DataFrame(CSV.File("personality_dataset.csv")) # โหลด dataset จาก csv
first(df, 5) # แสดง 5 แถวแรกของ DataFrame

In [ ]:
# อ่านชื่อคอลัมน์และขนาดของข้อมูล
println("Column names: ", names(df)) # ชื่อคอลัมน์
println("Data shape: ", size(df)) # ขนาดของข้อมูล
println("\nColumn descriptions:")
println("- Time_spent_Alone: เวลาอยู่คนเดียวต่อวัน (ชั่วโมง)")
println("- Stage_fear: กลัวการพูดต่อหน้าคนอื่นหรือไม่ (Yes/No)")
println("- Social_event_attendance: จำนวนงานสังคมที่เข้าร่วมต่อเดือน")
println("- Going_outside: จำนวนครั้งที่ออกไปข้างนอกต่อสัปดาห์")
println("- Drained_after_socializing: รู้สึกเหนื่อยล้าหลังการเข้าสังคมหรือไม่ (Yes/No)")
println("- Friends_circle_size: ขนาดกลุ่มเพื่อน")
println("- Post_frequency: จำนวนโพสต์บนโซเชียลมีเดียต่อสัปดาห์")
println("- Personality: ประเภทบุคลิกภาพ (Introvert/Extrovert)")

# 2. หาค่าเฉลี่ยของ Social_event_attendance และแสดง Density Plot

In [ ]:
# คัดกรองข้อมูลตามประเภทบุคลิกภาพ
introverts = filter(row -> row.Personality == "Introvert", df) # คัดกรอง Introvert
extroverts = filter(row -> row.Personality == "Extrovert", df) # คัดกรอง Extrovert

# คำนวณค่าเฉลี่ยของ Social_event_attendance สำหรับแต่ละกลุ่ม
mean_intro = mean(skipmissing(introverts.Social_event_attendance)) # ค่าเฉลี่ยของ Introvert
mean_extro = mean(skipmissing(extroverts.Social_event_attendance)) # ค่าเฉลี่ยของ Extrovert

println("ค่าเฉลี่ย Social_event_attendance:")
println("Introvert: ", round(mean_intro, digits=2))
println("Extrovert: ", round(mean_extro, digits=2))

In [ ]:
# สร้าง Density Plot
density(collect(skipmissing(introverts.Social_event_attendance)), 
        label="Introvert", linewidth=2, alpha=0.7) # กราฟของ Introvert
density!(collect(skipmissing(extroverts.Social_event_attendance)), 
         label="Extrovert", linewidth=2, alpha=0.7) # กราฟของ Extrovert

title!("Density Plot: Social Event Attendance")
xlabel!("Number of social events per month")
ylabel!("Density")

# เพิ่มเส้นแสดงค่าเฉลี่ย
vline!([mean_intro], label="Mean Introvert", linestyle=:dash, color=:blue) # เส้นเฉลี่ยของ Introvert
vline!([mean_extro], label="Mean Extrovert", linestyle=:dash, color=:orange) # เส้นเฉลี่ยของ Extrovert

savefig("density_social_events.png") # บันทึกกราฟเป็นไฟล์ png

# 3. แสดง 2D Scatter Plot ระหว่าง Friends_circle_size และ Time_spent_Alone

In [ ]:
# ลบแถวขาดข้อมูลในคอลัมน์ที่เราต้องการ
df_clean = dropmissing(df, [:Friends_circle_size, :Time_spent_Alone]) # ทำความสะอาดข้อมูล

# แยกกลุ่มข้อมูลตามประเภทบุคลิกภาพ
intro_clean = filter(row -> row.Personality == "Introvert", df_clean) # ข้อมูล Introvert ที่สมบูรณ์
extro_clean = filter(row -> row.Personality == "Extrovert", df_clean) # ข้อมูล Extrovert ที่สมบูรณ์

println("จำนวนข้อมูลหลังตัด missing values:")
println("Total: ", nrow(df_clean))
println("Introvert: ", nrow(intro_clean))
println("Extrovert: ", nrow(extro_clean))

In [ ]:
# สร้าง scatter plot
scatter(extro_clean.Friends_circle_size, extro_clean.Time_spent_Alone, 
        label="Extrovert", markersize=5, alpha=0.7, color=:orange) # กราฟของ Extrovert
scatter!(intro_clean.Friends_circle_size, intro_clean.Time_spent_Alone, 
         label="Introvert", markersize=5, alpha=0.7, color=:blue) # กราฟของ Introvert

title!("Friends Circle Size vs Time Spent Alone")
xlabel!("Friends Circle Size")
ylabel!("Time Spent Alone (hours/day)")

savefig("scatter_friends_vs_alone.png") # บันทึกกราฟเป็นไฟล์ png

# 4. แสดง Box Plot ของ Post_frequency

In [ ]:
# สร้าง box plot โดยใช้ StatsPlots
df_posts = dropmissing(df, :Post_frequency) # ลบข้อมูลที่ขาดใน Post_frequency

@df df_posts boxplot(:Personality, :Post_frequency, 
                     label=false, 
                     fillalpha=0.75,
                     linewidth=2) # สร้าง box plot

title!("Box Plot: Social Media Post Frequency by Personality Type")
xlabel!("Personality Type")
ylabel!("Number of posts per week")

savefig("boxplot_post_frequency.png") # บันทึกกราฟเป็นไฟล์ png

In [ ]:
# คำนวณและแสดงสถิติ
intro_posts = filter(row -> row.Personality == "Introvert", df_posts).Post_frequency # ข้อมูลโพสต์ของ Introvert
extro_posts = filter(row -> row.Personality == "Extrovert", df_posts).Post_frequency # ข้อมูลโพสต์ของ Extrovert

println("Post Frequency Statistics:")
println("\nIntrovert:")
println("  Mean: ", round(mean(intro_posts), digits=2))
println("  Median: ", median(intro_posts))
println("  Q1: ", quantile(intro_posts, 0.25))
println("  Q3: ", quantile(intro_posts, 0.75))

println("\nExtrovert:")
println("  Mean: ", round(mean(extro_posts), digits=2))
println("  Median: ", median(extro_posts))
println("  Q1: ", quantile(extro_posts, 0.25))
println("  Q3: ", quantile(extro_posts, 0.75))

# 5. สร้าง DataFrame ใหม่จากการสุ่ม 20 Introvert และ 20 Extrovert

In [ ]:
# ตั้งค่าสุ่มเพื่อให้ได้ผลลัพธ์ที่ซ้ำได้
Random.seed!(123) # กำหนดค่า seed ให้กับ random number

# ดึงข้อมูล introverts และ extroverts ทั้งหมด
all_introverts = filter(row -> row.Personality == "Introvert", df) # ข้อมูล Introvert ทั้งหมด
all_extroverts = filter(row -> row.Personality == "Extrovert", df) # ข้อมูล Extrovert ทั้งหมด

# สุ่มเลือก 20 จากแต่ละกลุ่ม
sampled_introverts = all_introverts[sample(1:nrow(all_introverts), 20, replace=false), :] # สุ่มเลือก Introvert
sampled_extroverts = all_extroverts[sample(1:nrow(all_extroverts), 20, replace=false), :] # สุ่มเลือก Extrovert

# รวมข้อมูลที่สุ่มได้ใน DataFrame ใหม่
new_df = vcat(sampled_introverts, sampled_extroverts) # รวมเป็น DataFrame ใหม่

println("New DataFrame created with 40 samples (20 Introvert + 20 Extrovert)") # แสดงผลการสร้าง DataFrame ใหม่

In [ ]:
# แสดงข้อมูลที่สุ่มได้
println("\nIntrovert samples (first 5):") 
println(first(sampled_introverts, 5)) # แสดง 5 แถวแรกของ Introvert

println("\nExtrovert samples (first 5):") 
println(first(sampled_extroverts, 5)) # แสดง 5 แถวแรกของ Extrovert

# ตรวจสอบจำนวน
println("\nVerification:")
println("Total samples: ", nrow(new_df))
println("Introverts: ", count(row -> row.Personality == "Introvert", eachrow(new_df)))
println("Extroverts: ", count(row -> row.Personality == "Extrovert", eachrow(new_df)))

# บันทึกเป็นไฟล์ CSV
CSV.write("sampled_personality_data.csv", new_df) # บันทึกข้อมูลที่สุ่มเป็นไฟล์ CSV
println("\nSampled data saved as 'sampled_personality_data.csv'")